In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pd.set_option('display.max_columns', None)
main_path = '../input/tabular-playground-series-jun-2022/'

df = pd.read_csv(main_path + 'data.csv', index_col='row_id')
submission = pd.read_csv(main_path + 'sample_submission.csv', index_col='row-col')

df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
#Extract the counts of null vaalues for each column
null_amts = pd.Series([df[col].isnull().sum() for col in df.columns], index=df.columns)
null_prob = null_amts.values / len(df)
null_values = pd.DataFrame({'Amount': null_amts, 'Probability': null_prob})
null_values = null_values.sort_values('Amount', ascending=False)
null_values.head(30)

In [ ]:
#si = SimpleImputer()
estimator = XGBRegressor(n_estimators=1000, 
                         learning_rate=0.2, 
                         max_depth=5, 
                         random_state=42, 
                         tree_method='gpu_hist')
ii = IterativeImputer(estimator=estimator, missing_values=np.nan, max_iter=10, random_state=42)
#knn = KNNImputer()

df[:] = ii.fit_transform(df)

In [ ]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = df.loc[row,col]

submission.to_csv('submission.csv')